# OptiGuide Example



Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization
The original system design for OptiGuide is 

![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## OptiGuide in Autogen

With autogen, we redesign the framework as follow.

Note that we have

![optiguide system](https://www.beibinli.com/docs/optiguide/autogen_optiguide_sys.png)

In [1]:
# %%capture # no printing logs here.
# # Install Gurobi for optimization
# %pip install gurobipy
# %pip install eventlet

# # Install FLAML for agents
# %pip install "flaml[optiguide]"

In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import re
import requests  # for loading the example source code
import openai
import os


# import flaml and autogen
from flaml import oai
from flaml.autogen.agent import Agent, GenericAgent
from flaml.autogen.agent.opti_guide import OptiGuideAgent

In [3]:
config_list = oai.config_list_gpt4_gpt35()
oai.ChatCompletion.start_logging()



In [4]:
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = "https://your.api.address.here.com"
openai.api_key = "<Your API Key goes here>"

openai.api_type = "azure"
openai.api_base = "https://msrcore.openai.azure.com"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("CORE_AZURE_KEY")

In [5]:
# Get the source code of our coffee example
code_url = "https://www.beibinli.com/docs/optiguide/coffee.py"

code =  requests.get(code_url).text

# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))


import time

from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
.
.
.

m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)




In [6]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```

"""

In [7]:
%%capture
agent = OptiGuideAgent(name="OptiGuide Coffee Example", 
                  source_code=code,
                  example_qa="")

user = GenericAgent("user", max_consecutive_auto_reply=0,
                         human_input_mode="NEVER", code_execution_config=False)


In [8]:
user.send("What if we prohibit shipping from supplier 1 to roastery 2?", agent)


user (to OptiGuide Coffee Example):

What if we prohibit shipping from supplier 1 to roastery 2?

--------------------------------------------------------------------------------
coder interpreter (to pencil):


Question: What if we prohibit shipping from supplier 1 to roastery 2?
Answer Code:


--------------------------------------------------------------------------------
pencil (to coder interpreter):

To prohibit shipping from supplier 1 to roastery 2, you can add a constraint to the model. Here's the code to do that:

```python
model.addConstr(x[('supplier1', 'roastery2')] == 0, "prohibit_sup1_to_roast2")
```

You should insert this code snippet right before the `# Optimize model` line in the original code. This constraint will set the flow between supplier 1 and roastery 2 to zero, effectively prohibiting any shipping between those two locations.

--------------------------------------------------------------------------------
model.addConstr(x[('supplier1', 'roastery2')] == 0, 

In [9]:
user.send("What is the impact of supplier1 being able to supply only half the quantity at present?", agent)


user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
coder interpreter (to pencil):


Question: What is the impact of supplier1 being able to supply only half the quantity at present?
Answer Code:


--------------------------------------------------------------------------------
pencil (to coder interpreter):

# First, update the capacity of supplier1 by halving the current value
new_capacity_supplier1 = capacity_in_supplier['supplier1'] / 2

# Then, update the supply constraint for supplier1
model.remove(model.getConstrByName("supply_supplier1"))
model.addConstr(sum(x[i] for i in shipping_cost_from_supplier_to_roastery.keys() if i[0] == 'supplier1') <= new_capacity_supplier1, "supply_supplier1")

# Solve the model again
model.optimize()

# If the model has an optimal solution, print the new objective value
if model.status == GRB.OPTI